# Imports

In [60]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from caveclient import CAVEclient
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from ccmodels.preprocessing.utils import layer_extractor
from ccmodels.preprocessing.connectomics import load_table, client_version
from standard_transform import minnie_transform_vx

tform_vx = minnie_transform_vx()

# Load data for model training

In [23]:
#Import functionally matched neurons and clean the table
client = client_version(661)
funcmatched = load_table(client, 'coregistration_manual_v3')


#Drop unlabelled neurons
funcmatched = funcmatched[funcmatched['pt_root_id']!=0]

#Drop neurons recorded in more than one scan
funcmatched = funcmatched.drop_duplicates(subset='pt_root_id', keep = 'first')

funcm = funcmatched[['id','pt_root_id',	'session','scan_idx','unit_id',	'pt_position']]

In [36]:
#Import brain area information and merge with the functional data
brainarea = pd.read_csv('../data/raw/area_membership.csv')
funcm_ba = brainarea.merge(funcm, on = ['session','scan_idx','unit_id'], how = 'inner')
funcm_ba.head()

,Unnamed: 0,session,scan_idx,unit_id,brain_area,id,pt_root_id,pt_position
0,647,4,7,648,RL,2043,864691135348268503,"[296208, 94688, 15803]"
1,661,4,7,662,V1,10173,864691135700505634,"[269248, 98832, 18382]"
2,664,4,7,665,RL,10871,864691135776919981,"[274864, 94064, 22046]"
3,670,4,7,671,V1,8088,864691135472842290,"[259936, 97360, 16974]"
4,681,4,7,682,V1,1480,864691135349237975,"[246448, 95856, 15873]"


In [37]:
target_classes = list(set(funcm_ba['brain_area']))

print(f'Target classes to categorise {len(target_classes)}: {target_classes}')

Target classes to categorise 4: ['RL', 'LM', 'AL', 'V1']


# Build train, validation, test and inference datasets

In [38]:
#Encoder for target categorical labels
encoder = LabelEncoder()
encoder.fit(['LM', 'V1', 'AL', 'RL'])


LabelEncoder()

In [39]:
#Check class distribution
funcm_ba.groupby('brain_area').count()

,Unnamed: 0,session,scan_idx,unit_id,id,pt_root_id,pt_position
brain_area,,,,,,,
AL,877,877,877,877,877,877,877
LM,30,30,30,30,30,30,30
RL,3184,3184,3184,3184,3184,3184,3184
V1,7960,7960,7960,7960,7960,7960,7960


In [40]:
#Define features and outputs
X = np.array(list(funcm_ba['pt_position']))
y = encoder.transform(funcm_ba['brain_area'])

#Split in trian and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Train and test models

In [41]:
#SVM with nonlinear kernel
svm = SVC(kernel = 'poly')
svm.fit(X_train, y_train)

SVC(kernel='poly')

In [42]:
pred = svm.predict(X_test)

In [43]:
print(classification_report(y_test, pred, target_names=encoder.classes_, zero_division= 0))

              precision    recall  f1-score   support

          AL       0.97      0.91      0.94       197
          LM       0.00      0.00      0.00         6
          RL       0.95      0.95      0.95       621
          V1       0.98      0.99      0.99      1587

    accuracy                           0.97      2411
   macro avg       0.73      0.72      0.72      2411
weighted avg       0.97      0.97      0.97      2411



Classes are quite imbalanced so lets try with a model that handles that better like random forest

In [44]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

RandomForestClassifier()

In [45]:
forestpreds = forest.predict(X_test)

In [46]:
print(classification_report(y_test, forestpreds, target_names=encoder.classes_))

              precision    recall  f1-score   support

          AL       0.97      0.93      0.95       197
          LM       1.00      0.83      0.91         6
          RL       0.97      0.98      0.98       621
          V1       1.00      1.00      1.00      1587

    accuracy                           0.99      2411
   macro avg       0.98      0.94      0.96      2411
weighted avg       0.99      0.99      0.99      2411



# Inference on inhibitory neurons

In [48]:
in_ex_neurs = load_table(client, 'baylor_log_reg_cell_type_coarse_v1')
in_ex_neurs.head(2)

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,classification_system,cell_type,pt_position,bb_start_position,bb_end_position
0,17115,2020-09-28 22:41:18.237823+00:00,t,268.646482,75934403318291307,864691135635239593,25718,2023-03-22 18:05:52.744496+00:00,t,17115,baylor_log_reg_cell_type_coarse,inhibitory,"[80992, 109360, 15101]","[nan, nan, nan]","[nan, nan, nan]"
1,17816,2020-09-28 22:42:54.932823+00:00,t,264.795587,75090047309035210,864691135618175635,25581,2023-03-22 18:05:52.650844+00:00,t,17816,baylor_log_reg_cell_type_coarse,inhibitory,"[74880, 110032, 16883]","[nan, nan, nan]","[nan, nan, nan]"


In [49]:
#Select only inhibitory neurons
inhibitory = in_ex_neurs[in_ex_neurs['cell_type'] == 'inhibitory']
#Only keep columns of interest
inhibitory_clean = inhibitory[['id', 'pt_root_id', 'pt_position', 'cell_type']].copy()
#Drop neurons recorded more than once
inhibitory_clean = inhibitory_clean.drop_duplicates(subset='pt_root_id', keep = 'first')

In [52]:
#Select only those cells that are neurons
nuc = load_table(client, 'aibs_soma_nuc_metamodel_preds_v117') #Table with neuronal information


neurs = nuc[nuc['classification_system']=='aibs_neuronal'][['id', 'classification_system']]
inhib_neur = inhibitory_clean.merge(neurs, on='id', how='inner')
inhib_neur  = inhib_neur[['id','pt_root_id','pt_position','cell_type']]

In [54]:
#Input features
X_inhib = np.array(list(inhib_neur['pt_position']))

In [55]:
#Predict the labels
preds_inhib = forest.predict(X_inhib)

In [56]:
#Convert them back to the ctaegorical lables
inhib_brain_areas = encoder.inverse_transform(preds_inhib)

In [57]:
#Format the predictions to the rest of the data
inhib_data_ba = inhib_neur.copy()
inhib_data_ba['brain_area'] = inhib_brain_areas

In [58]:
#Distributions look like what we saw in the previous data
inhib_data_ba.groupby('brain_area').count()

,id,pt_root_id,pt_position,cell_type
brain_area,,,,
AL,538,538,538,538
LM,37,37,37,37
RL,1216,1216,1216,1216
V1,2747,2747,2747,2747


In [63]:
#Add layer information
inhib_neurons_bal = layer_extractor(inhib_data_ba, tform_vx, column='pt_position')

In [65]:
inhib_neurons_bal.head(2)

,id,pt_root_id,pt_position,cell_type,brain_area,pial_distances,cortex_layer
0,25718,864691135635239593,"[80992, 109360, 15101]",inhibitory,V1,"[284.60979584385103, 67.34004818944197, 604.04...",L1
1,25581,864691135618175635,"[74880, 110032, 16883]",inhibitory,V1,"[260.02055322839834, 67.88703593921784, 675.32...",L1


In [66]:
#Save data
inhib_neurons_bal.to_pickle('../data/in_processing/inhibitory_nurons_bal.pkl')